In [121]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [122]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [123]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,10,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,50,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [124]:
# getting rid of non-ascii characters from the reviews as it breaks csv
try:
    df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
except:
    pass

In [125]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [126]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



16171
14668


In [127]:
df

beer_name beer_rating  \
0                   St. Peter's Ruby Red Ale         3.3   
1           Shepherd Neame Spitfire (Bottle)         2.2   
2                            Arbor Rocketman         3.8   
3     Shepherd Neame India Pale Ale (Bottle)         3.4   
4               Shepherd Neame Christmas Ale         3.3   
...                                      ...         ...   
1510       Wild Weather Damn Dead Grapefruit         3.5   
1511                Bas-Canada Petit Chicago         3.5   
1512                      ShinDigger Session         3.0   
1513                    Gallicus Double Aura         3.9   
1514        Liebharts Residenz "Scotch" Bier         3.2   

                     brewery_location                      brewery_name  \
0                     Bungay, Suffolk               St. Peter's Brewery   
1                     Faversham, Kent                    Shepherd Neame   
2            Bristol, Gloucestershire                        Arbor Ales   
3                     Faversham, Kent                    Shepherd Neame   
4                     Faversham, Kent                    Shepherd Neame   
...                               ...                               ...   
1510     Aldermaston Stoke, Berkshire                 Wild Weather Ales   
1511                 Gatineau, Quebec           Brasserie du Bas-Canada   
1512      Salford, Greater Manchester                ShinDigger Brewing   
1513                 Gatineau, Quebec     Brasserie Artisanale Gallicus   
1514  Detmold, North Rhine-Westphalia  Liebharts Privatbrauerei Detmold   

                                         review_content        reviewer_name  
0     Ruby color, offwhite coam. Aroma and taste: fl...            alex_leit  
1     Disappointing show from the lads at ‘Britain’s...  shivermetimber.gray  
2     Puszka. Mętne o żółtym kolorze. Piana biała,ob...              MarcinG  
3     Bottle. Amber with bronze color and white foam...            alex_leit  
4     Bottle. Aroma: English hops and malts with tof...            alex_leit  
...                                                 ...                  ...  
1510  Pale golden, cloudy with no head or lacing. Ar...       SampleLegion47  
1511  Can, 950 ml, purchased at Broue Ha Ha on March...                  GRM  
1512  Pale golden, slightly hazy with short-lasting ...       SampleLegion47  
1513  Can: Poured a cloudy/hazy yellow/beige color a...               Boutip  
1514  Bottle 0844/1142. Peated nose. The beer starts...             Erzengel  

[14668 rows x 6 columns]

In [128]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [129]:
from selenium import webdriver
import time

In [130]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [131]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 769 new beers to search for!


In [132]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [133]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

return_string
0    Outer Range Two Pound Tent\n🇺🇸 IIPA DIPA - Imp...
1    Caffrey's Premium Irish Ale\n🇬🇧 Bitter - Ordin...
2    Union Bremer Kräusen\n🇩🇪 Zwickelbier / Kellerb...
3    Other Half Double Sunset - Ethiopian Yirgz Cof...
4    Dorking Pilcrow Pale\n🇬🇧 Blonde Ale / Golden A...
..                                                 ...
764  Fourpure Nightcap\n🇬🇧 Stout - Imperial • 9.0%\...
765  Früh Radler\n🇩🇪 Radler / Shandy • 2.5%\n5\n65\...
766  Vitamin Sea All Together\n🇺🇸 IPA - Hazy / NEIP...
767  Vocation Bocktoberfest\n🇬🇧 Bock - Doppelbock •...
768  St. Peter's Raven's Orchard\n🇬🇧 Apple Cider • ...

[769 rows x 1 columns]

In [134]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [135]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [136]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

return_string  \
9    Hairless Dog N.A. Black Ale\n🇺🇸 Low / No Alcoh...   
57   Riedenburger Ur-Helles Alkoholfrei\n🇩🇪 Low / N...   
127  Singlecut This Ain't No Foolin' Around DDH IPA...   
208  Urban South Paradise Pork\n🇺🇸 Pale Lager - Nor...   
224  Club Beer (Liberia)\n🇱🇷 Pale Lager - North Am....   
271  Central Waters Brewer's Reserve Bourbon Barrel...   
340  No Boats on Sunday Mixed Berry Cider\n🇨🇦 Apple...   
360  Sawdust City / Block 3 Nouveau Monde Bière de ...   
432  Wilde Rose Kellerbier\n🇩🇪 Zwickelbier / Keller...   
518  New Glarus Thumbprint Series Nectar Ale\n🇺🇸 La...   
538  Cycle Thursday - 2020\n🇺🇸 Stout - Imperial Fla...   
655  Freigeist / Stillwater / Weingut Völker Scheur...   
732  Camerons 12 Mile India Pale Lager\n🇨🇦 Pale Lag...   

                                            name_found flag  \
9                          Hairless Dog N.A. Black Ale   🇺🇸   
57                  Riedenburger Ur-Helles Alkoholfrei   🇩🇪   
127     Singlecut This Ain't No Foolin' Around DDH IPA   🇺🇸   
208                          Urban South Paradise Pork   🇺🇸   
224                                Club Beer (Liberia)   🇱🇷   
271  Central Waters Brewer's Reserve Bourbon Barrel...   🇺🇸   
340               No Boats on Sunday Mixed Berry Cider   🇨🇦   
360  Sawdust City / Block 3 Nouveau Monde Bière de ...   🇨🇦   
432                              Wilde Rose Kellerbier   🇩🇪   
518            New Glarus Thumbprint Series Nectar Ale   🇺🇸   
538                              Cycle Thursday - 2020   🇺🇸   
655  Freigeist / Stillwater / Weingut Völker Scheur...   🇩🇪   
732                  Camerons 12 Mile India Pale Lager   🇨🇦   

                                       type  abv rb_overall_score  \
9              Low / No Alcohol Beer - Dark  NaN                -   
57             Low / No Alcohol Beer - Pale  NaN                -   
127                          IPA - Flavored  NaN                -   
208   Pale Lager - North Am. / Light / Rice  NaN                -   
224   Pale Lager - North Am. / Light / Rice  0.0                2   
271                  Scotch Ale / Wee Heavy  0.0               99   
340                  Apple Cider - Flavored  NaN                -   
360           Saison / Farmhouse / Grisette  0.0                -   
432     Zwickelbier / Kellerbier / Landbier  0.0               76   
518                        Lambic - Fruited  NaN                -   
538               Stout - Imperial Flavored  NaN                -   
655                        Sour / Wild Beer  NaN                -   
732              Pale Lager - India / Hoppy  0.0               68   

    rb_style_score rb_user_rating rb_rating_number  
9                -           2.00                1  
57               -           2.90                1  
127              -           3.50                1  
208              -           3.00                1  
224             33           1.78               34  
271            100           3.95              278  
340              -           3.10                1  
360              -           3.84                5  
432             97           3.44               54  
518              -           3.78                4  
538              -           4.30                1  
655              -           3.42                4  
732             60           3.35               41

In [137]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [138]:
beer_df

return_string flag  \
0    St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...   🇬🇧   
1    Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...   🇬🇧   
2    Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...   🇬🇧   
3    Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...   🇬🇧   
4    Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...   🇬🇧   
..                                                 ...  ...   
432  Wilde Rose Kellerbier\n🇩🇪 Zwickelbier / Keller...   🇩🇪   
518  New Glarus Thumbprint Series Nectar Ale\n🇺🇸 La...   🇺🇸   
538  Cycle Thursday - 2020\n🇺🇸 Stout - Imperial Fla...   🇺🇸   
655  Freigeist / Stillwater / Weingut Völker Scheur...   🇩🇪   
732  Camerons 12 Mile India Pale Lager\n🇨🇦 Pale Lag...   🇨🇦   

                                     type  abv rb_overall_score  \
0               Bitter - Ordinary / Best   4.3               49   
1               Bitter - Ordinary / Best   4.5               34   
2                                    IPA   6.0               95   
3                          IPA - English   6.1               46   
4                   Strong Ale - English   7.0               45   
..                                    ...  ...              ...   
432   Zwickelbier / Kellerbier / Landbier  0.0               76   
518                      Lambic - Fruited  NaN                -   
538             Stout - Imperial Flavored  NaN                -   
655                      Sour / Wild Beer  NaN                -   
732            Pale Lager - India / Hoppy  0.0               68   

    rb_style_score rb_user_rating rb_rating_number  \
0               95           3.25              415   
1               59           3.05             1083   
2               97           3.74               59   
3               47           3.22              420   
4               46           3.18              409   
..             ...            ...              ...   
432             97           3.44               54   
518              -           3.78                4   
538              -           4.30                1   
655              -           3.42                4   
732             60           3.35               41   

                                            name_found  
0                             St. Peter's Ruby Red Ale  
1                     Shepherd Neame Spitfire (Bottle)  
2                                      Arbor Rocketman  
3               Shepherd Neame India Pale Ale (Bottle)  
4                         Shepherd Neame Christmas Ale  
..                                                 ...  
432                              Wilde Rose Kellerbier  
518            New Glarus Thumbprint Series Nectar Ale  
538                              Cycle Thursday - 2020  
655  Freigeist / Stillwater / Weingut Völker Scheur...  
732                  Camerons 12 Mile India Pale Lager  

[9757 rows x 9 columns]

In [139]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 47 new beers that were not found or returned errors. They will be added to the no search list.


In [140]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)